In [1]:
install.packages('rpart')
install.packages('rpart.plot')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("rpart.plot"):
“installation of package ‘rpart.plot’ had non-zero exit status”

In [5]:
# Load needed packages
library(caTools)
library(ROCR)
library(rpart)
library(rpart.plot)

In [7]:
df <- read.csv("Datasets/HEMQ.csv",na.strings=c("NA",""))
df$BAD <- factor(df$BAD)

In [8]:
set.seed(101)

In [9]:
# Data partition
sample <- sample.split(df$BAD, SplitRatio = 0.7)

In [11]:
# Create Training Data
df.train = subset(df, sample == TRUE)
# Create Validation Data
df.valid = subset(df, sample == FALSE)

In [12]:
# Build a decision tree model
tree.model <- rpart(BAD~.,method="class",data=df.train)

In [13]:
# Display decision tree results
printcp(tree.model)


Classification tree:
rpart(formula = BAD ~ ., data = df.train, method = "class")

Variables actually used in tree construction:
[1] CLAGE   DEBTINC DELINQ  DEROG   MORTDUE NINQ    VALUE   YOJ    

Root node error: 832/4172 = 0.19942

n= 4172 

        CP nsplit rel error  xerror     xstd
1 0.031250      0   1.00000 1.00000 0.031020
2 0.025240      5   0.84014 0.88702 0.029623
3 0.022837      6   0.81490 0.87861 0.029512
4 0.018029      7   0.79207 0.86538 0.029336
5 0.013221      8   0.77404 0.85938 0.029255
6 0.012019      9   0.76082 0.85938 0.029255
7 0.010000     10   0.74880 0.86659 0.029352


In [14]:
#Tree plot
prp(tree.model,type=2,extra=106)

In [15]:
#Predict the default probabilities based on the validataion dataset
pred.probabilities <- predict(tree.model,df.valid)
pred.probabilities <- as.data.frame(pred.probabilities)

In [16]:
#Turn the default probabilities to binary, threshold is 0.5
joiner <- function(x){
  if (x>=0.5){
    return('Yes')
  }else{
    return("No")
  }
}

In [17]:
pred.probabilities$Result <- sapply(pred.probabilities$`1`,joiner)

In [18]:
# Create the confusion matrix
print("The confusion matrix is:")
print(table(pred.probabilities$Result,df.valid$BAD))

[1] "The confusion matrix is:"
     
         0    1
  No  1371  232
  Yes   60  125


In [19]:
# Create the ROC curve
pred <- prediction(pred.probabilities$`1`,df.valid$BAD)
perf <- performance(pred, "tpr", "fpr")
plot(perf)
abline(a=0,b=1)

In [20]:
# Calculate and print AUC value
auc <- performance(pred, measure="auc")
auc <- auc@y.values[[1]]
print(paste("AUC for the baseline regression model is:", auc))

[1] "AUC for the baseline regression model is: 0.690120912096495"
